In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

class SVM:
    def __init__(self, learning_rate=0.001, lambda_l1=0.01, lambda_l2=0.01, n_iters=1000):
        self.learning_rate = learning_rate
        self.lambda_l1 = lambda_l1
        self.lambda_l2 = lambda_l2
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Initialize weights and bias
        self.w = np.zeros(n_features)
        self.b = 0

        # Convert labels to -1 and 1
        y_ = np.where(y <= 0, -1, 1)

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    # Only apply L2 regularization
                    self.w -= self.learning_rate * (2 * self.lambda_l2 * self.w)
                else:
                    # Apply both L1 and L2 regularization
                    self.w -= self.learning_rate * (2 * self.lambda_l2 * self.w - np.dot(x_i, y_[idx])) + self.learning_rate * self.lambda_l1 * np.sign(self.w)
                    self.b -= self.learning_rate * y_[idx]

    def predict(self, X):
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)

# Load dataset (assuming the dataset is a CSV file)
data = pd.read_csv('phishing_dataset.csv')

# Separate features and labels
X = data.drop('label', axis=1)  # Assuming the label column is named 'label'
y = data['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and train the custom SVM classifier
svm_classifier = SVM(learning_rate=0.001, lambda_l1=0.01, lambda_l2=0.01, n_iters=1000)
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test)

# Evaluate model performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))